In [ ]:
# Uncomment to enable debugging
# TEST_SLUG = 'woodland-tribe'
if 'TEST_SLUG' in vars() and TEST_SLUG is not None:
    import yaml

    slug = TEST_SLUG

    with open('./params.yaml') as f:
        params = yaml.load(f, yaml.SafeLoader)['events'][slug]

    spektrix_event_ids = params['spektrix_event_ids']
    venue_postcode = params['venue_postcode']
    rosterfy_event_ids = params['rosterfy_event_ids']
    project_ids = params['project_ids']
    venue_ids = params['venue_ids']


In [ ]:
spektrix_event_ids: str
venue_postcode: str
rosterfy_event_ids: str
project_ids: str
venue_ids: str
slug: str

In [ ]:
assert 'slug' in vars() and slug is not None, "'slug' not defined'"

In [ ]:
import json
from datetime import datetime

from utils.paths import SITE
from utils.process.event import Programme, Sustainability, Tickets, Volunteers

In [ ]:
OUTPUT_DIR = SITE / 'events' / slug / '_data/event'
PROGRAMME_DIR = OUTPUT_DIR / 'programme'
PROGRAMME_DIR.mkdir(exist_ok=True, parents=True)
TICKETS_DIR = OUTPUT_DIR / 'tickets'
TICKETS_DIR.mkdir(exist_ok=True, parents=True)
VOLUNTEERS_DIR = OUTPUT_DIR / 'volunteers'
SUSTAINABILITY_DIR = OUTPUT_DIR / 'sustainability'

In [ ]:
if project_ids or venue_ids:
    programme = Programme(project_ids=project_ids.split(), venue_ids=venue_ids.split())
    with open(PROGRAMME_DIR / 'summary.json', 'w') as f:
        json.dump(programme.summarise(), f, indent=2)
    (
        programme.events
        .aggregate(['venue_name', 'project_name'], {
            'start_date': ('date', min),
            'end_date': ('date', max),
            'event_count': (len),
            'audience': ('audience', sum),
            'participants': ('schedule_participants_community', sum),
        })
        .sort('start_date')
        .convert(['start_date', 'end_date'], lambda x: x.isoformat())
        .tojson(PROGRAMME_DIR / 'total_by_venue.json', indent=2)
    )

In [ ]:
if spektrix_event_ids:
    tickets = Tickets(spektrix_event_ids.split())
    tickets.detailed().tocsv(TICKETS_DIR / 'by_geography.csv')
    tickets.summarise().tocsv(TICKETS_DIR / 'summary.csv')
    tickets.types().tocsv(TICKETS_DIR / 'by_type.csv')

In [ ]:
if rosterfy_event_ids:
    VOLUNTEERS_DIR.mkdir(exist_ok=True, parents=True)
    volunteers = Volunteers(str(rosterfy_event_ids).split())
    volunteers.summarise().tocsv(VOLUNTEERS_DIR / 'summary.csv')

In [ ]:
if project_ids:
    SUSTAINABILITY_DIR.mkdir(exist_ok=True, parents=True)
    sustainability = Sustainability(project_ids.split()).summarise()
    sustainability.tocsv(SUSTAINABILITY_DIR / 'breakdown.csv')
    sustainability.select(lambda r: r.impact_type == 'ALL' and r.scope == 'ALL').cut('tCO2e').tojson(SUSTAINABILITY_DIR / 'summary.json', lines=True)